In [ ]:
#beginning

In [1]:
%matplotlib inline
import matplotlib
import os              # This provides several system utilities
import pandas as pd    # This is the workhorse of data munging in Python
import numpy as np     # This is for general numerical operations 
import seaborn as sns  # This allows us to efficiently and beautifully plot
import networkx as nx  
import osmnx as ox
import statsmodels.api as sm
import matplotlib.pyplot as plt

# display settings (optional)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

In [2]:
#for the import process, next step is to create a string variable containing the directory of the file and a file name variable
infodir = r'C:\Users\EPRO\OneDrive - Wageningen University & Research\Course Channel Metropolitan Challenges 2025 - Paper Group 6 Parking space vs Food Space'
fname = infodir + r'\data\buurten_filtered.csv'

gpdf01 = pd.read_csv(fname)
#this is to verify
gpdf01.head()

,OBJECTNUMMER,BRT,SDC,Periode,Dagdeel,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT
0,117.0,AA01 Planciusbuurt-Noord,A,202403-202404,doordeweeks avond,55.0,52.0,3.0,0.0,55.0,95.0,100.0,"POLYGON((4.8857454 52.3881665,4.8844789 52.388...","POLYGON((52.3881665 4.8857454,52.3883785 4.884...",4.884390,52.387023
1,118.0,AA02 Westelijke eilanden,A,202403-202404,doordeweeks avond,313.0,287.0,9.0,0.0,296.0,92.0,95.0,"POLYGON((4.8910092 52.3884398,4.8859714 52.388...","POLYGON((52.3884398 4.8910092,52.3881287 4.885...",4.887548,52.385401
2,119.0,AA03 Westerdokseiland,A,202403-202404,doordeweeks avond,165.0,76.0,0.0,0.0,76.0,46.0,46.0,"POLYGON((4.8950776 52.3886909,4.8923612 52.388...","POLYGON((52.3886909 4.8950776,52.3885233 4.892...",4.894474,52.384417
3,120.0,AA04 Planciusbuurt-Zuid,A,202403-202404,doordeweeks avond,20.0,15.0,1.0,0.0,16.0,75.0,80.0,"POLYGON((4.8839232 52.385505,4.884072 52.38566...","POLYGON((52.385505 4.8839232,52.3856666 4.8840...",4.882867,52.384947
4,121.0,AA05 Haarlemmerbuurt-West,A,202403-202404,doordeweeks avond,148.0,141.0,4.0,0.0,145.0,95.0,98.0,"POLYGON((4.8839232 52.385505,4.8835118 52.3849...","POLYGON((52.385505 4.8839232,52.3849538 4.8835...",4.886377,52.383013


In [3]:
#here i am dropping all entries that do not have "doordeweeks avond" in "Dagdeel"
#and checking how many I have left
gpdf01_1 = gpdf01[gpdf01['Dagdeel'] == 'doordeweeks avond']
before = len(gpdf01)
after = len(gpdf01_1)
print(f"Rows before: {before}, after: {after}, dropped: {before - after}")

Rows before: 1447, after: 518, dropped: 929


In [4]:
#here i am purging all rows with "netto_pkd" and "bruto_pkd" higher than 150% because that looks like missing and damaged data
#that would otherwise interfere with our next process
gpdf01_2 = gpdf01_1[(gpdf01_1["Netto_PKD"] <= 150) & (gpdf01_1["Bruto_PKD"] <= 150)]
gpdf01_2 = gpdf01_2.sort_values(by=["Bruto_PKD", "Netto_PKD"], ascending=[False, False])
gpdf01_2.head(10)

,OBJECTNUMMER,BRT,SDC,Periode,Dagdeel,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT
26,143.0,AE02 Oude Kerk e.o.,A,202301-202302,doordeweeks avond,8.0,7.0,2.0,0.0,9.0,88.0,112.0,"POLYGON((4.8994004 52.3747475,4.8992273 52.374...","POLYGON((52.3747475 4.8994004,52.3747876 4.899...",4.896203,52.373811
97,384.0,TH04 K-buurt-Zuidwest,T,202406,doordeweeks avond,358.0,340.0,57.0,0.0,397.0,95.0,111.0,"POLYGON((4.9803772 52.3155975,4.9801227 52.315...","POLYGON((52.3155975 4.9803772,52.3155437 4.980...",4.979595,52.313042
98,385.0,TH05 K-buurt-Zuidoost,T,202406,doordeweeks avond,327.0,302.0,46.0,0.0,348.0,92.0,106.0,"POLYGON((4.9859463 52.3173694,4.9854452 52.317...","POLYGON((52.3173694 4.9859463,52.3172527 4.985...",4.984753,52.314853
87,374.0,TE03 G-buurt-Noord,T,202406,doordeweeks avond,320.0,306.0,29.0,0.0,335.0,96.0,105.0,"POLYGON((4.9714808 52.3253526,4.9706196 52.325...","POLYGON((52.3253526 4.9714808,52.3250574 4.970...",4.970181,52.323912
89,376.0,TF02 Grunder/Koningshoef,T,202406,doordeweeks avond,425.0,416.0,24.0,0.0,440.0,98.0,104.0,"POLYGON((4.9769028 52.3238067,4.977699 52.3229...","POLYGON((52.3238067 4.9769028,52.3229333 4.977...",4.980608,52.321355
157,563.0,EG02 GWL-terrein,E,202409-202410,doordeweeks avond,38.0,39.0,0.0,0.0,39.0,103.0,103.0,"POLYGON((4.8694074 52.3855428,4.8668659 52.385...","POLYGON((52.3855428 4.8694074,52.3854968 4.866...",4.868767,52.383811
281,1032.0,FB07 Bakemabuurt,F,202409-202411,doordeweeks avond,533.0,513.0,24.0,0.0,537.0,96.0,101.0,"POLYGON((4.8093999 52.3793915,4.8094205 52.379...","POLYGON((52.3793915 4.8093999,52.379517 4.8094...",4.805575,52.378331
70,357.0,TA03 Amstel III deel A/B-Noord,T,202406,doordeweeks avond,4.0,4.0,0.0,0.0,4.0,100.0,100.0,"POLYGON((4.9475058 52.3108069,4.9463557 52.312...","POLYGON((52.3108069 4.9475058,52.3120613 4.946...",4.946693,52.305855
139,545.0,EB08 Zeeheldenbuurt,E,202409-202410,doordeweeks avond,212.0,205.0,7.0,0.0,212.0,97.0,100.0,"POLYGON((4.8856593 52.3907556,4.8855179 52.390...","POLYGON((52.3907556 4.8856593,52.3904709 4.885...",4.888207,52.389442
0,117.0,AA01 Planciusbuurt-Noord,A,202403-202404,doordeweeks avond,55.0,52.0,3.0,0.0,55.0,95.0,100.0,"POLYGON((4.8857454 52.3881665,4.8844789 52.388...","POLYGON((52.3881665 4.8857454,52.3883785 4.884...",4.884390,52.387023
